In [7]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score
from answer_judge import load_judge

In [8]:
data_reader=ReadData('/root/autodl-fs/hallscope/ScienceQA/train')
data=data_reader.read_all()

random.shuffle(data)

# Set the seed np.random.seed(42) # Generate random numbers 
random_numbers = np.random.rand(5)

In [9]:
df=pd.DataFrame(data)

In [10]:
df.head()

,question,answer,most_likely
0,[Question:Complete the sentence.\nSewing an ap...,"[tensor(1), tensor(0), tensor(0), tensor(2)]",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...
1,[Question:What do these two changes have in co...,"[tensor(0), tensor(0)]",{'embedding': [[[ 0.00308228 0.01495361 -0.01...
2,[Question:Which of these states is farthest no...,"[tensor(0), tensor(1), tensor(0), tensor(1)]",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...


In [13]:
df.iloc[1]['question']

['Question:What do these two changes have in common?\ntearing a piece of paper\nbreaking a piece of glass\nAnswer the Question with following choices.\nChoices: A.Both are only physical changes. B.Both are chemical changes. C.Both are caused by cooling. D.Both are caused by heating.\n',
 'Question:Which of these organisms contains matter that was once part of the phytoplankton?\nAnswer the Question with following choices.\nChoices: A.black rockfish B.sea otter\n']

In [14]:
df.iloc[1]['most_likely']['response']

['A.Both are only physical changes.', 'A. black rockfish']

In [5]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()
embeddings=np.concatenate(embeddings,axis=0)
embeddings.shape

(3573, 33, 4096)

In [6]:
gt_label=df['is_safe'].to_list()
gt_label=np.concatenate(gt_label,axis=0)
print(f'There are {len(gt_label)} questions. {sum(gt_label==1)} answers are safe and {sum(gt_label==0)} answers are unsafe')

There are 3573 questions. 1800 answers are safe and 1773 answers are unsafe


In [7]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
resps=[]
for x in responses:
    resps+=x

In [8]:
resps[:5]

['Yes', 'Yes', 'Yes', 'Yes', 'Yes']

In [9]:
is_hall=[]
judge=load_judge(dataset='MMSafety',prompt_type='mq')
for i,x in enumerate(resps):
    is_hall.append(judge.check(x,gt_label[i]))
is_hall=np.array(is_hall)

In [10]:
print(f'There are {len(is_hall)} responses. {sum(is_hall==1)} answer correctly and {sum(is_hall==0)} answer wrongly')

There are 3573 responses. 1862 answer correctly and 1711 answer wrongly


In [11]:
length=len(is_hall)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 1000
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_hall[test_idxs]
hall_label_wild = is_hall[train_idxs]
hall_label_val = is_hall[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:1858
validset length:1000
testset length:715
trainset postive: 967 negative: 891
valset postive: 531 negative: 469
testset postive: 364 negative: 351


In [12]:
all_embeddings = embeddings
all_embeddings.shape

(3573, 33, 4096)

In [13]:
embedding_train = all_embeddings[train_idxs][:, 1:, :]
embedding_val = all_embeddings[val_idxs][:, 1:, :]
embedding_test = all_embeddings[test_idxs][:, 1:, :]

In [14]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

Grid Search for Best 'n_components' & 'layer'

In [15]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [16]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(12,1,-1),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:12,layer:0 Score: 0.48150691257192646
found best n_components:12,layer:1 Score: 0.5814510980207919
found best n_components:12,layer:2 Score: 0.5910520039030032
found best n_components:12,layer:3 Score: 0.602287191965917
found best n_components:12,layer:4 Score: 0.6688068936993804
found best n_components:12,layer:5 Score: 0.7216781307345436
found best n_components:12,layer:7 Score: 0.7618927155987616
found best n_components:12,layer:12 Score: 0.7822590036098764
found best n_components:12,layer:26 Score: 0.7849493452832688


  9%|▉         | 1/11 [00:02<00:22,  2.29s/it]

found best n_components:12,layer:29 Score: 0.7900288709800473


...

 73%|███████▎  | 8/11 [00:16<00:06,  2.08s/it]

found best n_components:5,layer:30 Score: 0.8139729118732408


 82%|████████▏ | 9/11 [00:18<00:04,  2.05s/it]

found best n_components:4,layer:29 Score: 0.8281995992595538


100%|██████████| 11/11 [00:22<00:00,  2.06s/it]


In [ ]:
best_paras

{'n_components': 4, 'layer': 29, 'best_score': np.float64(0.8281995992595538)}

In [18]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [19]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(hall_label_val)

In [20]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.5161463939720129 AUROC:0.7872439203014867


In [21]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.6503496503496503 AUROC:0.8025343602266679


Train Linear Prob to Get Better Results

In [25]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [26]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [28]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)} f1:{f1_score(y_test,pred_test)}')

test set acc: 0.509090909090909 AUROC:0.7749639961178423 f1:0.6746987951807228
